In [155]:
import cv2 as cv
import numpy as np
import pymysql
import math
from collections import Counter

In [156]:
# get the database
def select_star(_bam,_cam,_dam,_cbm,_dbm):
    db = pymysql.connect('127.0.0.1', 'root', 'QqQQ513203745', 'test2')
    cursor = db.cursor()
    sql = 'SELECT a.mag amag, b.mag bmag, c.mag cmag, d.mag dmag FROM star3 a, star3 b, star3 c , star3 d where b.mag-a.mag='+str(_bam)+'and c.mag-a.mag='+str(_cam)+'and d.mag-a.mag='+str(_dam)+'and c.mag-b.mag='+str(_cbm)+'and d.mag-b.mag='+str(_dbm)+'limit 3'
    print(sql)
    try:
        cursor.execute(sql)
        results = cursor.fetchall()
        for row in results:
            amag = row[0]
            bmag = row[1]
            cmag = row[2]
            dmag = row[3]
            dic = {'amag':amag, 'bamg':bmag, 'cmag':cmag, 'dmag':dmag}
            print(dic)
    except Exception as e:
        print(e)
        print('error: unable to fetch data')
    db.close()

In [157]:
def count_area(cnt):
    area = cv.contourArea(cnt)
    return area

In [158]:
def custom_threshold(picname):
    image = cv.imread(picname)
    #Image Binaryzation
    gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    h, w = gray.shape[:2]
    m = np.reshape(gray, [1,w*h])
    mean = 120
    ret, binary = cv.threshold(gray, mean, 255, cv.THRESH_BINARY)
    binary, contours, hierarchy = cv.findContours(binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    print('length', len(contours))
    area = []
    cont_arr = []
    for i in range(len(contours)):
        # Get the radius of every star
        (x, y), radius = cv.minEnclosingCircle(contours[i])
        if((h-y)>(h/5)):
            area.append(cv.contourArea(contours[i]))
            cont_arr.append(contours[i])
    cont_arr.sort(key = count_area, reverse=True)
    radius_arr = []
    for i in range(0, len(cont_arr)):
        if i == 4:
            break;
        (x,y),radius = cv.minEnclosingCircle(cont_arr[i])
        radius_arr.append(radius)
        center = (int(x), int(y))
        radius = int(radius)
        # the brightest star in the image
        cv.circle(image, center, radius, (0, 255, 0), 2)
    
    scale_one=math.pow(radius_arr[0], 2)/math.pow(radius_arr[1], 2)
    scale_two=math.pow(radius_arr[0], 2)/math.pow(radius_arr[2], 2)
    scale_three=math.pow(radius_arr[0], 2)/math.pow(radius_arr[3], 2)
    scale_four=math.pow(radius_arr[1], 2)/math.pow(radius_arr[2], 2)
    scale_five=math.pow(radius_arr[1], 2)/math.pow(radius_arr[3], 2)
    scale_six=math.pow(radius_arr[2], 2)/math.pow(radius_arr[3], 2)
    
    bam=round(math.log(scale_one,2.512),2)
    cam=round(math.log(scale_two,2.512),2)
    dam=round(math.log(scale_three,2.512),2)
    cbm=round(math.log(scale_four,2.512),2)
    dbm=round(math.log(scale_five,2.512),2)
    print(select_star(bam,cam,dam,cbm,dbm))
    
    cv.imwrite(picname+"_gray.jpg", image)        

In [159]:
custom_threshold("Night_Sky_1.jpg")

length 2542
SELECT a.mag amag, b.mag bmag, c.mag cmag, d.mag dmag FROM star3 a, star3 b, star3 c , star3 d where b.mag-a.mag=-0.0and c.mag-a.mag=0.61and d.mag-a.mag=0.64and c.mag-b.mag=0.61and d.mag-b.mag=0.64limit 3
{'amag': Decimal('0.9800000'), 'bamg': Decimal('0.9800000'), 'cmag': Decimal('1.5900000'), 'dmag': Decimal('1.6200000')}
{'amag': Decimal('1.5900000'), 'bamg': Decimal('1.5900000'), 'cmag': Decimal('2.2000000'), 'dmag': Decimal('2.2300000')}
{'amag': Decimal('1.5900000'), 'bamg': Decimal('1.5900000'), 'cmag': Decimal('2.2000000'), 'dmag': Decimal('2.2300000')}
None


In [121]:
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style, quantity_support
plt.style.use(astropy_mpl_style)
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz

In [138]:
def locate_star(star_ra, star_dec):
    star = SkyCoord(star_ra, star_dec, frame='icrs', unit='deg')
    earth_location = EarthLocation(lat=42.1*u.deg, lon=74*u.deg)
    utcoffset = -4*u.hour  
    time = Time('2017-10-12 01:11:00') - utcoffset
    star_alt_azi = star.transform_to(AltAz(obstime=time,location=earth_location))
    print("star's Altitude = {0.alt:.3}".format(star_alt_azi))
    print("star's Azimuth = {0.alt:.3}".format(star_alt_azi))